In [30]:
import qsharp

We are using 8 different configurations, 4 for 12 qubits and 4 for 24 qubits:
* 12 Qubits - Default Setup
* 12 Qubits - 1 Distillation Factory
* 12 Qubits - Trapped Ion Qubits
* 12 Qubits - Majorana Qubits

For 24 qubits (this file):
* 24 Qubits - Default Setup (Baseline with default qubit type and default distillation factories)
* 24 Qubits - 1 Distillation Factory
* 24 Qubits - Trapped Ion Qubits
* 24 Qubits - Majorana Qubits

The main comparisons will be between the same number of qubits and different configs, but baseline 12 and 24 will be compared as well to see how the resources scale

In [31]:
%%qsharp
//Grover's Search taken from https://learn.microsoft.com/en-us/azure/quantum/tutorial-qdk-grovers-search?tabs=tabid-vscode#the-grovers-algorithm
// This is for 24 qubits

import Microsoft.Quantum.Convert.*;
import Microsoft.Quantum.Math.*;
import Microsoft.Quantum.Arrays.*;
import Microsoft.Quantum.Measurement.*;
import Microsoft.Quantum.Diagnostics.*;
import Microsoft.Quantum.ResourceEstimation.*;


@EntryPoint()
operation Main() : Result[] {
    let nQubits = 24;
    let iterations = CalculateOptimalIterations(nQubits);
    Message($"Number of iterations: {iterations}");

    // Use Grover's algorithm to find a particular marked state.
    let results = GroverSearch(nQubits, iterations, ReflectAboutMarked);
    return results;
}

operation GroverSearch(
    nQubits : Int,
    iterations : Int,
    phaseOracle : Qubit[] => Unit) : Result[] {

    use qubits = Qubit[nQubits];

    PrepareUniform(qubits);

    for _ in 1..iterations {
        // if (BeginEstimateCaching("GroverAESExample.EstimateIteration", SingleVariant())) {
            phaseOracle(qubits);
            ReflectAboutUniform(qubits);
            //EstimatingAES(qubits);
        //     EndEstimateCaching();
        // }
    }

    // Measure and return the answer.
    return MResetEachZ(qubits);
}

function CalculateOptimalIterations(nQubits : Int) : Int {
    if nQubits > 63 {
        fail "This sample supports at most 63 qubits.";
    }
    let nItems = 1 <<< nQubits; // 2^nQubits
    let angle = ArcSin(1. / Sqrt(IntAsDouble(nItems)));
    let iterations = Round(0.25 * PI() / angle - 0.5);
    return iterations;
}

operation ReflectAboutMarked(inputQubits : Qubit[]) : Unit {
    Message("Reflecting about marked state...");
    use outputQubit = Qubit();
    within {
        // We initialize the outputQubit to (|0⟩ - |1⟩) / √2, so that
        // toggling it results in a (-1) phase.
        X(outputQubit);
        H(outputQubit);
        // Flip the outputQubit for marked states.
        // Here, we get the state with alternating 0s and 1s by using the X
        // operation on every other qubit.
        for q in inputQubits[...2...] {
            X(q);
        }
        
    } apply {
        Controlled X(inputQubits, outputQubit);
        EstimatingAES(inputQubits);
    }
}

operation PrepareUniform(inputQubits : Qubit[]) : Unit is Adj + Ctl {
    for q in inputQubits {
        H(q);
    }
}

operation ReflectAboutAllOnes(inputQubits : Qubit[]) : Unit {
    Controlled Z(Most(inputQubits), Tail(inputQubits));
}

operation EstimatingAES(inputQubits : Qubit[]) : Unit {
    //Scaled values from scaling.py
    //Based on the values from Jang et al.
    AccountForEstimates(
        [TCount(91094), 
        AuxQubitCount(5031), 
        RotationCount(17931), 
        RotationDepth(13231), 
        MeasurementCount(12)],
        PSSPCLayout(), inputQubits);
}
operation ReflectAboutUniform(inputQubits : Qubit[]) : Unit {
    within {
        // Transform the uniform superposition to all-zero.
        Adjoint PrepareUniform(inputQubits);
        // Transform the all-zero state to all-ones
        for q in inputQubits {
            X(q);
        }
    } apply {
        // Now that we've transformed the uniform superposition to the
        // all-ones state, reflect about the all-ones state, then let the
        // within/apply block transform us back.
        ReflectAboutAllOnes(inputQubits);
    }


}

In [42]:
result_24_default = qsharp.estimate("Main()")
# 24 qubits, automatic number of distillation factories, and qubit_gate_ns_e3
result_24_default

Runtime,5 hours,"Total runtime This is a runtime estimate for the execution time of the algorithm. In general, the execution time corresponds to the duration of one logical cycle (12,400 nanosecs) multiplied by the 1,415,007,864 logical cycles to run the algorithm. If however the duration of a single T factory (here: 156,800 nanosecs) is larger than the algorithm runtime, we extend the number of logical cycles artificially in order to exceed the runtime of a single T factory."
rQOPS,831.85M,"Reliable quantum operations per second The value is computed as the number of logical qubits after layout (10,315) (with a logical error rate of 2.28e-17) multiplied by the clock frequency (80,645.16), which is the number of logical cycles per second."
Physical qubits,20.71M,"Number of physical qubits This value represents the total number of physical qubits, which is the sum of 19,825,430 physical qubits to implement the algorithm logic, and 881,280 physical qubits to execute the T factories that are responsible to produce the T states that are consumed by the algorithm."
Logical algorithmic qubits,10.31k,"Number of logical qubits for the algorithm after layout Laying out the logical qubits in the presence of nearest-neighbor constraints requires additional logical qubits. In particular, to layout the $Q_{\rm alg} = 5,056$ logical qubits in the input algorithm, we require in total $2 \cdot Q_{\rm alg} + \lceil \sqrt{8 \cdot Q_{\rm alg}}\rceil + 1 = 10,315$ logical qubits."
Algorithmic depth,1.42G,"Number of logical cycles for the algorithm To execute the algorithm using _Parallel Synthesis Sequential Pauli Computation_ (PSSPC), operations are scheduled in terms of multi-qubit Pauli measurements, for which assume an execution time of one logical cycle. Based on the input algorithm, we require one multi-qubit measurement for the 176,904 single-qubit measurements, the 57,666,096 arbitrary single-qubit rotations, and the 292,958,304 T gates, three multi-qubit measurements for each of the 144,720 CCZ and 0 CCiX gates in the input program, as well as 25 multi-qubit measurements for each of the 42,550,896 non-Clifford layers in which there is at least one single-qubit rotation with an arbitrary angle rotation."
Logical depth,1.42G,"Number of logical cycles performed This number is usually equal to the logical depth of the algorithm, which is 1,415,007,864. However, in the case in which a single T factory is slower than the execution time of the algorithm, we adjust the logical cycle depth to exceed the T factory's execution time."
Clock frequency,80.64k,Number of logical cycles per second This is the number of logical cycles that can be performed within one second. The logical cycle time is 12 microsecs.
Number of T states,1.74G,"Number of T states consumed by the algorithm To execute the algorithm, we require one T state for each of the 292,958,304 T gates, four T states for each of the 144,720 CCZ and 0 CCiX gates, as well as 25 for each of the 57,666,096 single-qubit rotation gates with arbitrary angle rotation."
Number of T factories,16,"Number of T factories capable of producing the demanded 1,735,189,584 T states during the algorithm's runtime The total number of T factories 16 that are executed in parallel is computed as $\left\lceil\dfrac{\text{T states}\cdot\text{T factory duration}}{\text{T states per T factory}\cdot\text{algorithm runtime}}\right\rceil = \left\lceil\dfrac{1,735,189,584 \cdot 156,800\;\text{ns}}{1 \cdot 17,546,097,513,600\;\text{ns}}\right\rceil$"
Number of T factory invocations,108.45M,"Number of times all T factories are invoked In order to prepare the 1,735,189,584 T states, the 16 copies of the T factory are repeatedly invoked 108,449,349 times."
Physical algorithmic qubits,19.83M,"Number of physical qubits for the algorithm after layout The 19,825,430 are the product of the 10,315 logical qubits after layout and the 1,922 physical qubits that encode a single logical qubit."


In [33]:
result_24_default['jobParams']
# 24 qubits, automatic number of distillation factories, and qubit_gate_ns_e3

{'qecScheme': {'name': 'surface_code',
  'errorCorrectionThreshold': 0.01,
  'crossingPrefactor': 0.03,
  'logicalCycleTime': '(4 * twoQubitGateTime + 2 * oneQubitMeasurementTime) * codeDistance',
  'physicalQubitsPerLogicalQubit': '2 * codeDistance * codeDistance',
  'maxCodeDistance': 50},
 'errorBudget': 0.001,
 'qubitParams': {'instructionSet': 'GateBased',
  'name': 'qubit_gate_ns_e3',
  'oneQubitMeasurementTime': '100 ns',
  'oneQubitGateTime': '50 ns',
  'twoQubitGateTime': '50 ns',
  'tGateTime': '50 ns',
  'oneQubitMeasurementErrorRate': 0.001,
  'oneQubitGateErrorRate': 0.001,
  'twoQubitGateErrorRate': 0.001,
  'tGateErrorRate': 0.001,
  'idleErrorRate': 0.001},
 'constraints': {'maxDistillationRounds': 3},
 'estimateType': 'singlePoint'}

In [34]:
result_24_default.diagram.space

In [35]:
result_24_default.diagram.time

In [36]:
results_24_tfac_one= qsharp.estimate("Main()", 
params={'constraints': {'maxTFactories': 1}})
results_24_tfac_one
# 24 qubits, 1 distillation factory limited, and qubit_gate_ns_e3


Runtime,3 days,"Total runtime This is a runtime estimate for the execution time of the algorithm. In general, the execution time corresponds to the duration of one logical cycle (13,200 nanosecs) multiplied by the 1,415,007,864 logical cycles to run the algorithm. If however the duration of a single T factory (here: 156,800 nanosecs) is larger than the algorithm runtime, we extend the number of logical cycles artificially in order to exceed the runtime of a single T factory."
rQOPS,781.44M,"Reliable quantum operations per second The value is computed as the number of logical qubits after layout (10,315) (with a logical error rate of 3.00e-18) multiplied by the clock frequency (75,757.58), which is the number of logical cycles per second."
Physical qubits,22.52M,"Number of physical qubits This value represents the total number of physical qubits, which is the sum of 22,466,070 physical qubits to implement the algorithm logic, and 55,080 physical qubits to execute the T factories that are responsible to produce the T states that are consumed by the algorithm."
Logical algorithmic qubits,10.31k,"Number of logical qubits for the algorithm after layout Laying out the logical qubits in the presence of nearest-neighbor constraints requires additional logical qubits. In particular, to layout the $Q_{\rm alg} = 5,056$ logical qubits in the input algorithm, we require in total $2 \cdot Q_{\rm alg} + \lceil \sqrt{8 \cdot Q_{\rm alg}}\rceil + 1 = 10,315$ logical qubits."
Algorithmic depth,1.42G,"Number of logical cycles for the algorithm To execute the algorithm using _Parallel Synthesis Sequential Pauli Computation_ (PSSPC), operations are scheduled in terms of multi-qubit Pauli measurements, for which assume an execution time of one logical cycle. Based on the input algorithm, we require one multi-qubit measurement for the 176,904 single-qubit measurements, the 57,666,096 arbitrary single-qubit rotations, and the 292,958,304 T gates, three multi-qubit measurements for each of the 144,720 CCZ and 0 CCiX gates in the input program, as well as 25 multi-qubit measurements for each of the 42,550,896 non-Clifford layers in which there is at least one single-qubit rotation with an arbitrary angle rotation."
Logical depth,20.61G,"Number of logical cycles performed This number is usually equal to the logical depth of the algorithm, which is 1,415,007,864. However, in the case in which a single T factory is slower than the execution time of the algorithm, we adjust the logical cycle depth to exceed the T factory's execution time."
Clock frequency,75.76k,Number of logical cycles per second This is the number of logical cycles that can be performed within one second. The logical cycle time is 13 microsecs.
Number of T states,1.74G,"Number of T states consumed by the algorithm To execute the algorithm, we require one T state for each of the 292,958,304 T gates, four T states for each of the 144,720 CCZ and 0 CCiX gates, as well as 25 for each of the 57,666,096 single-qubit rotation gates with arbitrary angle rotation."
Number of T factories,1,"Number of T factories capable of producing the demanded 1,735,189,584 T states during the algorithm's runtime The total number of T factories 1 that are executed in parallel is computed as $\left\lceil\dfrac{\text{T states}\cdot\text{T factory duration}}{\text{T states per T factory}\cdot\text{algorithm runtime}}\right\rceil = \left\lceil\dfrac{1,735,189,584 \cdot 156,800\;\text{ns}}{1 \cdot 272,077,726,773,600\;\text{ns}}\right\rceil$"
Number of T factory invocations,1.74G,"Number of times all T factories are invoked In order to prepare the 1,735,189,584 T states, the 1 copies of the T factory are repeatedly invoked 1,735,189,584 times."
Physical algorithmic qubits,22.47M,"Number of physical qubits for the algorithm after layout The 22,466,070 are the product of the 10,315 logical qubits after layout and the 2,178 physical qubits that encode a single logical qubit."


In [37]:
results_24_tfac_one.diagram.space

In [38]:
results_24_trapped_ion= qsharp.estimate("Main()", 
params={
                    "qubitParams": {
                        "name": "qubit_gate_us_e3"
                    }})
results_24_trapped_ion
# 12 qubits, default distillation factory, and trapped ion qubit gate (qubit_gate_us_e3)


Runtime,305 days,"Total runtime This is a runtime estimate for the execution time of the algorithm. In general, the execution time corresponds to the duration of one logical cycle (18,600,000 nanosecs) multiplied by the 1,415,007,864 logical cycles to run the algorithm. If however the duration of a single T factory (here: 195,000,000 nanosecs) is larger than the algorithm runtime, we extend the number of logical cycles artificially in order to exceed the runtime of a single T factory."
rQOPS,554.57k,"Reliable quantum operations per second The value is computed as the number of logical qubits after layout (10,315) (with a logical error rate of 2.28e-17) multiplied by the clock frequency (53.76), which is the number of logical cycles per second."
Physical qubits,20.15M,"Number of physical qubits This value represents the total number of physical qubits, which is the sum of 19,825,430 physical qubits to implement the algorithm logic, and 325,000 physical qubits to execute the T factories that are responsible to produce the T states that are consumed by the algorithm."
Logical algorithmic qubits,10.31k,"Number of logical qubits for the algorithm after layout Laying out the logical qubits in the presence of nearest-neighbor constraints requires additional logical qubits. In particular, to layout the $Q_{\rm alg} = 5,056$ logical qubits in the input algorithm, we require in total $2 \cdot Q_{\rm alg} + \lceil \sqrt{8 \cdot Q_{\rm alg}}\rceil + 1 = 10,315$ logical qubits."
Algorithmic depth,1.42G,"Number of logical cycles for the algorithm To execute the algorithm using _Parallel Synthesis Sequential Pauli Computation_ (PSSPC), operations are scheduled in terms of multi-qubit Pauli measurements, for which assume an execution time of one logical cycle. Based on the input algorithm, we require one multi-qubit measurement for the 176,904 single-qubit measurements, the 57,666,096 arbitrary single-qubit rotations, and the 292,958,304 T gates, three multi-qubit measurements for each of the 144,720 CCZ and 0 CCiX gates in the input program, as well as 25 multi-qubit measurements for each of the 42,550,896 non-Clifford layers in which there is at least one single-qubit rotation with an arbitrary angle rotation."
Logical depth,1.42G,"Number of logical cycles performed This number is usually equal to the logical depth of the algorithm, which is 1,415,007,864. However, in the case in which a single T factory is slower than the execution time of the algorithm, we adjust the logical cycle depth to exceed the T factory's execution time."
Clock frequency,54,Number of logical cycles per second This is the number of logical cycles that can be performed within one second. The logical cycle time is 19 millisecs.
Number of T states,1.74G,"Number of T states consumed by the algorithm To execute the algorithm, we require one T state for each of the 292,958,304 T gates, four T states for each of the 144,720 CCZ and 0 CCiX gates, as well as 25 for each of the 57,666,096 single-qubit rotation gates with arbitrary angle rotation."
Number of T factories,13,"Number of T factories capable of producing the demanded 1,735,189,584 T states during the algorithm's runtime The total number of T factories 13 that are executed in parallel is computed as $\left\lceil\dfrac{\text{T states}\cdot\text{T factory duration}}{\text{T states per T factory}\cdot\text{algorithm runtime}}\right\rceil = \left\lceil\dfrac{1,735,189,584 \cdot 195,000,000\;\text{ns}}{1 \cdot 26,319,146,270,400,000\;\text{ns}}\right\rceil$"
Number of T factory invocations,133.48M,"Number of times all T factories are invoked In order to prepare the 1,735,189,584 T states, the 13 copies of the T factory are repeatedly invoked 133,476,122 times."
Physical algorithmic qubits,19.83M,"Number of physical qubits for the algorithm after layout The 19,825,430 are the product of the 10,315 logical qubits after layout and the 1,922 physical qubits that encode a single logical qubit."


In [39]:
results_24_trapped_ion.diagram.space

In [40]:
results_24_majorana= qsharp.estimate("Main()", 
params={
                    "qubitParams": {
                        "name": "qubit_maj_ns_e6"
                    },
                    "qecScheme": {
                        "name": "floquet_code"
                    }})
results_24_majorana
# 12 qubits, default distillation factory, and qubit_maj_ns_e6 with floquet code error correction


Runtime,50 mins,"Total runtime This is a runtime estimate for the execution time of the algorithm. In general, the execution time corresponds to the duration of one logical cycle (2,100 nanosecs) multiplied by the 1,415,007,864 logical cycles to run the algorithm. If however the duration of a single T factory (here: 37,200 nanosecs) is larger than the algorithm runtime, we extend the number of logical cycles artificially in order to exceed the runtime of a single T factory."
rQOPS,4.91G,"Reliable quantum operations per second The value is computed as the number of logical qubits after layout (10,315) (with a logical error rate of 2.28e-17) multiplied by the clock frequency (476,190.48), which is the number of logical cycles per second."
Physical qubits,2.88M,"Number of physical qubits This value represents the total number of physical qubits, which is the sum of 2,516,860 physical qubits to implement the algorithm logic, and 366,080 physical qubits to execute the T factories that are responsible to produce the T states that are consumed by the algorithm."
Logical algorithmic qubits,10.31k,"Number of logical qubits for the algorithm after layout Laying out the logical qubits in the presence of nearest-neighbor constraints requires additional logical qubits. In particular, to layout the $Q_{\rm alg} = 5,056$ logical qubits in the input algorithm, we require in total $2 \cdot Q_{\rm alg} + \lceil \sqrt{8 \cdot Q_{\rm alg}}\rceil + 1 = 10,315$ logical qubits."
Algorithmic depth,1.42G,"Number of logical cycles for the algorithm To execute the algorithm using _Parallel Synthesis Sequential Pauli Computation_ (PSSPC), operations are scheduled in terms of multi-qubit Pauli measurements, for which assume an execution time of one logical cycle. Based on the input algorithm, we require one multi-qubit measurement for the 176,904 single-qubit measurements, the 57,666,096 arbitrary single-qubit rotations, and the 292,958,304 T gates, three multi-qubit measurements for each of the 144,720 CCZ and 0 CCiX gates in the input program, as well as 25 multi-qubit measurements for each of the 42,550,896 non-Clifford layers in which there is at least one single-qubit rotation with an arbitrary angle rotation."
Logical depth,1.42G,"Number of logical cycles performed This number is usually equal to the logical depth of the algorithm, which is 1,415,007,864. However, in the case in which a single T factory is slower than the execution time of the algorithm, we adjust the logical cycle depth to exceed the T factory's execution time."
Clock frequency,476.19k,Number of logical cycles per second This is the number of logical cycles that can be performed within one second. The logical cycle time is 2 microsecs.
Number of T states,1.74G,"Number of T states consumed by the algorithm To execute the algorithm, we require one T state for each of the 292,958,304 T gates, four T states for each of the 144,720 CCZ and 0 CCiX gates, as well as 25 for each of the 57,666,096 single-qubit rotation gates with arbitrary angle rotation."
Number of T factories,22,"Number of T factories capable of producing the demanded 1,735,189,584 T states during the algorithm's runtime The total number of T factories 22 that are executed in parallel is computed as $\left\lceil\dfrac{\text{T states}\cdot\text{T factory duration}}{\text{T states per T factory}\cdot\text{algorithm runtime}}\right\rceil = \left\lceil\dfrac{1,735,189,584 \cdot 37,200\;\text{ns}}{1 \cdot 2,971,516,514,400\;\text{ns}}\right\rceil$"
Number of T factory invocations,78.87M,"Number of times all T factories are invoked In order to prepare the 1,735,189,584 T states, the 22 copies of the T factory are repeatedly invoked 78,872,254 times."
Physical algorithmic qubits,2.52M,"Number of physical qubits for the algorithm after layout The 2,516,860 are the product of the 10,315 logical qubits after layout and the 244 physical qubits that encode a single logical qubit."


In [41]:
results_24_majorana.diagram.space